In [386]:
%pylab
import talib
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as dates
import pandas as pd
from sqlalchemy import create_engine 
from __future__ import division


engine = create_engine('postgresql://postgres@localhost:5432/postgres')

instrument = 'EUR_USD'

#fromDate = '2016-11-06 00:00:00+00'
#toDate = '2017-03-10 18:40:00+00'
fromDate = '2016-03-11 00:00:00+00'
toDate = '2016-11-05 00:00:00+00'
period = '1h'
fromtbl = 'oanda_%s_m1' % (instrument)

# for ask_*
win_agg_group_sql_ask = '''
  with intervals as (
  select start, start + interval '%s' as end
  from generate_series('%s', '%s', interval '%s') as start
  )
  select distinct
  intervals.start as dt,
  min(ask_l) over w as low,
  max(ask_h) over w as high,
  first_value(ask_o) over w as open,
  last_value(ask_c) over w as close,
  sum(v) over w as volume
  from
  intervals
  join %s tmb on
  tmb.dt >= intervals.start and
  tmb.dt < intervals.end
  window w as (partition by intervals.start)
  order by intervals.start;
''' % (period,fromDate,toDate,period,fromtbl)

# for ask_bid
win_agg_group_sql = '''
  with intervals as (
  select start, start + interval '%s' as end
  from generate_series('%s', '%s', interval '%s') as start
  )
  select distinct
  intervals.start as dt,
  min(bid_l) over w as low,
  max(bid_h) over w as high,
  first_value(bid_o) over w as open,
  last_value(bid_c) over w as close,
  sum(v) over w as volume
  from
  intervals
  join %s tmb on
  tmb.dt >= intervals.start and
  tmb.dt < intervals.end
  window w as (partition by intervals.start)
  order by intervals.start;
''' % (period,fromDate,toDate,period,fromtbl)

orig_df = pd.read_sql_query(win_agg_group_sql, 
                       con=engine)
#orig_df.dt = pd.to_datetime(orig_df.dt,utc=True).dt.tz_convert('Europe/Moscow')
orig_df.dt = pd.to_datetime(orig_df.dt,utc=True)
orig_df.set_index(['dt'])
orig_df['dt']=orig_df.dt.dt.strftime('%m/%d/%Y %H:%M:%S')

orig_df['atr14']=talib.ATR(orig_df['high'].values,
                            orig_df['low'].values,
                            orig_df['close'].values,
                            14)
    
wov_df=orig_df.drop(['volume'],axis=1)


Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [387]:

def min_resampler(array_like):
    a=array_like.to_frame(name='tmp')
    #print a
    #print 'min:%f' % a['tmp'].min()
    dt_str = pd.to_datetime(a[(a['tmp']==a['tmp'].min())].index).strftime("%H")
    #print type(dt_str)
    #print dt_str.astype(int)
    #print dt_str.dtype
    if( dt_str.data ):
        return dt_str 
    else:
        return -1

    

def max_resampler(array_like):
    a=array_like.to_frame(name='tmp')
    #print a
    #print 'min:%f' % a['tmp'].min()
    dt_str = pd.to_datetime(a[(a['tmp']==a['tmp'].max())].index).strftime("%H")
    #print type(dt_str)
    #print dt_str.astype(int)
    #print dt_str.dtype
    if( dt_str.data ):
        return dt_str 
    else:
        return -1

In [388]:
#atr_df = wov_df.set_index(pd.DatetimeIndex(wov_df['dt'])).atr14.resample('24H').apply(min_resampler)
atr_df = wov_df.set_index(pd.DatetimeIndex(wov_df['dt'])).atr14.resample('24H').apply(max_resampler)
atr_df = atr_df[atr_df != -1]
#atr_df.head(40)

In [390]:
def get_stats(group):
    return {'count': group.count()}

bins = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10 ,11,12,13,14,15,16,17,18,19,20,21,22,23]
g_atr = pd.cut(atr_df, bins)
grouped = atr_df.groupby(g_atr)
bin_counts = grouped.apply(get_stats).unstack()
bin_counts.index =  [ '1','2','3','4','5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18','19','20','21','22','23' ]
bin_counts.index.name = 'time hour/PDT'
bin_counts.plot(kind='bar', alpha=0.5, rot=0)

In [ ]:
![atr_max_pst](images/yaktocat.png)